## Simplex algoriythm
An algorithm for optimization<br>
sense is maximize here -> maximizing the objective

In [1]:
import numpy as np

In [2]:
def mat_modifications(objective, constraints, solution):
    arr = np.vstack((-1 * objective, constraints))
    left_arr = np.zeros(arr.shape[0])
    left_arr[0] = 1
    left_arr = left_arr.reshape(-1,1)
    arr = np.hstack((left_arr, arr))
    s_arr = np.identity(constraints.shape[0])
    upp_arr = np.zeros(constraints.shape[0])
    s_arr = np.vstack((np.zeros(constraints.shape[0]), s_arr))
    arr = np.hstack((arr, s_arr))
    arr = np.hstack((arr, solution.reshape(-1,1)))
    return arr

In [6]:
objective = np.array([5,4])
constraints = np.array([[6,4],[1,2],[-1,1],[0,1]])
solutions = np.array([0,24,6,1,2])
A = mat_modifications(objective, constraints, solutions)
A

array([[ 1., -5., -4.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  6.,  4.,  1.,  0.,  0.,  0., 24.],
       [ 0.,  1.,  2.,  0.,  1.,  0.,  0.,  6.],
       [ 0., -1.,  1.,  0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  1.,  2.]])

#### First form form of row finding<br>
You should PAY ATTENTION to large values of your matrix or use the second form

In [35]:
# def col_finder(_matrix):
#     matrix = np.copy(_matrix)
#     col_number = np.argmin(matrix[0])

def row_finder(col_number, _matrix):
    matrix = np.copy(_matrix)
    col = matrix[1:, col_number]
    col = np.where(col>0, col, 10**-6)
    sol = matrix[1:, -1]
    return np.argmin(sol/(col))+1    #1 row was deleted at first and now we are adding that

row_finder(1, A)

1

### Second form
If U want to use first form and don't wanna contribute with divide by zero error, don't run this

In [32]:
# def col_finder(_matrix):
#     matrix = np.copy(_matrix)
#     col_number = np.argmin(matrix[0])

def row_finder(col_number, _matrix):
    matrix = np.copy(_matrix)
    col = matrix[1:, col_number]
    sol = matrix[1:, -1]
    # print(col, sol)
    res = sol/col
    res = np.where(res>=0, res, np.inf)
    return np.argmin(res)+1    #1 row was deleted at first and now we are adding that

row_finder(1, A)

C:\Users\ToTo\AppData\Local\Temp\ipykernel_8752\2656738885.py:10: RuntimeWarning: divide by zero encountered in divide
  res = sol/col


1

In [36]:
# row operation in simplex method, approximatly simillar to elimination
def row_opp(col_number, row_number, _matrix):
    matrix = np.copy(_matrix)    # gives us a deep copy of enered matrix
    arr = matrix[row_number] / matrix[row_number, col_number]
    matrix[row_number] = arr
    i = 0

    while i < matrix.shape[0]:
        if i == row_number:
            i += 1
            continue

        mat_arr = matrix[i]
        
        if mat_arr[col_number] != 0:
            # print(mat_arr[col_number])
            new_arr = mat_arr - arr * mat_arr[col_number]
            matrix[i] = new_arr
            i += 1
        else:
            i += 1
            continue

    return np.round(matrix, 3)   # U can remove `round` to get more accurate answer

b = row_opp(1, 1, A)
b

array([[ 1.   ,  0.   , -0.667,  0.833,  0.   ,  0.   ,  0.   , 20.   ],
       [ 0.   ,  1.   ,  0.667,  0.167,  0.   ,  0.   ,  0.   ,  4.   ],
       [ 0.   ,  0.   ,  1.333, -0.167,  1.   ,  0.   ,  0.   ,  2.   ],
       [ 0.   ,  0.   ,  1.667,  0.167,  0.   ,  1.   ,  0.   ,  5.   ],
       [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  2.   ]])

In [37]:
def symplex(_matrix, n_var): #n_var = constraints.shape[1]     number of variables
    matrix = np.copy(_matrix)
    if min(matrix[0, 1:n_var+1]) >= 0:
        return matrix
    
    else:
        # print(matrix)
        col_number = np.argmin(matrix[0, 1:n_var+1]) + 1
        row_number = row_finder(col_number, matrix)
        # print(col_number, row_number)
        matrix = row_opp(col_number, row_number, matrix)
        # print('here in iter the matrix is\n',matrix )
        return symplex(matrix, n_var)

x = symplex(A, 2)
x

array([[ 1.   ,  0.   ,  0.   ,  0.749,  0.5  ,  0.   ,  0.   , 21.001],
       [ 0.   ,  1.   ,  0.   ,  0.251, -0.5  ,  0.   ,  0.   ,  2.999],
       [ 0.   ,  0.   ,  1.   , -0.125,  0.75 ,  0.   ,  0.   ,  1.5  ],
       [ 0.   ,  0.   ,  0.   ,  0.376, -1.251,  1.   ,  0.   ,  2.499],
       [ 0.   ,  0.   ,  0.   ,  0.125, -0.75 ,  0.   ,  1.   ,  0.5  ]])